In [1]:
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np
import ConfigParser
from nltk import ngrams
import hashlib



In [2]:
g65_configFileLocation = './config'
Config = ConfigParser.ConfigParser()
Config.read(g65_configFileLocation)
def ConfigSectionMap(section):
    dict1 = {}
    options = Config.options(section)
    for option in options:
        try:
            dict1[option] = Config.get(section, option)
            if dict1[option] == -1:
                DebugPrint("skip: %s" % option)
        except:
            print("exception on %s!" % option)
            dict1[option] = None
    return dict1




In [3]:
#reading our configured values
msd_subset_path = ConfigSectionMap("MainSection")['datasetlocation']
msd_code_path = ConfigSectionMap("MainSection")['codelocation']
g65_selectedFeatures =ConfigSectionMap("MainSection")['features'].split("<g65SEP>")
g65_n = int(ConfigSectionMap("MainSection")['numberofsongstoprocess'])  # number of songs to process 
g65_k = int(ConfigSectionMap("MainSection")['k'])# number of shingles(grams)
g65_hashalgorithm = ConfigSectionMap("MainSection")['hashalgorithm']#hashing algorithm try print(hashlib.algorithms_available) print(hashlib.algorithms_guaranteed) to show all of the available ones


signatureMatrix = {}# a dictionary of arrays, with unique shingles as keys each shingle has as value a list 
#the list at each element will have 0 or 1 depending on the corresponding song at that index has that shingle or not :D
songsIntId=[-1] # will be used to generate the integer index id for our songs
songsIds = {} #key= the dataset key for the song , #value = our numerical index that can be used for its list position for a shingle in the dictionary signatureMatrix

In [4]:
print g65_n

2


In [5]:

msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path'

In [6]:
#msd_code_path=os.path.join(msd_subset_path,'MSongsDB')
assert os.path.isdir(msd_code_path),'wrong path'

In [7]:
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )

In [8]:
import hdf5_getters as GETTERS


In [9]:
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

In [10]:
limitCounter = [0]
def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    """
    From a base directory, go through all subdirectories,
    find all files with the given extension, apply the
    given function 'func' to all of them.
    INPUT
       basedir  - base directory of the dataset
       func     - function to apply to all filenames
       ext      - extension, .h5 by default
    """
    # iterate over all files in all subdirectories
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        # apply function to all files, quit when we exeed the number of songs to precess n
        for f in files :
            #print GETTERS.get_song_id(f)
            if limitCounter[0]== g65_n:
                return
            else:
                func(f) 
            

In [11]:
# we define the function to apply to all files
def func_to_get_artist_name(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    all_artist_names.append( artist_name )
    h5.close()
    
def func_to_get_Selected_features(filename):
    """
    Given a song file name, 
    this function reads the song file,
    gets from it the selected features in our config file,
    concatinates them together,
    then adds them as a separate n shingles to our global list of songs
    
    
    """
    
    h5 = GETTERS.open_h5_file_read(filename)
    song =''
    songsIntId[0]+=1
    for feature in g65_selectedFeatures:
        song+=((str(getattr(GETTERS, 'get_'+feature)(h5))+' '))#concatinating the features into one space separated string
    songGrams = ngrams(song.split(),g65_k)#creating n shingles out of the space separated File output =list of n shingles elements
    song=[]
    for grams in songGrams:# for each ngrams element in the list 
        hashedgram = ( getattr(hashlib, g65_hashalgorithm)(''.join(grams)).hexdigest())
        songDatasetId = GETTERS.get_song_id(h5)
        songsIds[songDatasetId] = songsIntId[0]
        if not signatureMatrix.has_key(hashedgram):
            signatureMatrix[hashedgram]=[0]*(g65_n-1) # the first time for that shingle - not sure why -1, need to fix it #$%$
        signatureMatrix[hashedgram].insert(songsIds[songDatasetId],1)
        #song.append(hashedgram)#I hash them then together and append them to song
    
    #songs.append(song)#append the hashed shingles to songs
    limitCounter[0]+=1# a counter for how many songs are processed so far
    h5.close()
    

In [12]:
print g65_n

2


In [21]:
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=func_to_get_Selected_features)
t2 = time.time()
print 'all of the selected features were returned in:',strtimedelta(t1,t2)

all of the selected features were returned in: 0:00:00.003098


In [22]:
#print songsIds
#TODO min hashing
print signatureMatrix 


{'f188c17763e14981a4f1be72db7366fc': [1, 0], '542e18daec9c661fb3e90f724bf07644': [0, 1], '9f9184562fed30cbe342f7f1913e9da8': [0, 1], '9720dc655924528b17bcda523c3e5d48': [0, 1], 'd110b1c5f868beca0a062ec69c67e4d0': [0, 1], 'c69f8f1e8f206f353dc1e4677db220af': [1, 0], 'b30de68786aa715de1d22664c12f06e8': [1, 0], '39cf27f2bed68ae51cc735958a487de4': [0, 1], 'f7ddd489ab0a82567b241b05971cbdb3': [1, 0], '10725539ae2780e2d173f30cea8d79c5': [0, 1], '587d0dd40b83d6f5628583a8a1e77a34': [1, 0], 'bd3bfeb3ff74837ffa9118727edade82': [1, 0], 'de74dbb9893732ce0748511c826f572f': [1, 0], '9b4bc4b195aee738870ccd783034d50f': [0, 1]}


In [416]:
dir(GETTERS)


['__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__warningregistry__',
 'get_analysis_sample_rate',
 'get_artist_7digitalid',
 'get_artist_familiarity',
 'get_artist_hotttnesss',
 'get_artist_id',
 'get_artist_latitude',
 'get_artist_location',
 'get_artist_longitude',
 'get_artist_mbid',
 'get_artist_mbtags',
 'get_artist_mbtags_count',
 'get_artist_name',
 'get_artist_playmeid',
 'get_artist_terms',
 'get_artist_terms_freq',
 'get_artist_terms_weight',
 'get_audio_md5',
 'get_bars_confidence',
 'get_bars_start',
 'get_beats_confidence',
 'get_beats_start',
 'get_danceability',
 'get_duration',
 'get_end_of_fade_in',
 'get_energy',
 'get_key',
 'get_key_confidence',
 'get_loudness',
 'get_mode',
 'get_mode_confidence',
 'get_num_songs',
 'get_release',
 'get_release_7digitalid',
 'get_sections_confidence',
 'get_sections_start',
 'get_segments_confidence',
 'get_segments_loudness_max',
 'get_segments_loudness_max_time',
 'get_segments_loudness_start',
 'get_seg

In [34]:
GETTERS.get_song_id('~/mmds-project-dependencies/MillionSongSubset/data/A/A/A/TRAAABD128F429CF47.h5')

AttributeError: 'str' object has no attribute 'root'